In [1]:
#Imports
#Imports
import numpy as np
import pandas as pd
import seaborn as sns
import ast, json
import math
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
from statsmodels.stats import weightstats as stests
import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import tree
from yellowbrick.regressor import ResidualsPlot
from math import sqrt
from sklearn.model_selection import KFold, cross_val_score
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn import svm
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn import datasets
 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble.forest import RandomForestRegressor

from sklearn.preprocessing import scale, normalize
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
%matplotlib inline

In [2]:
#Useful methods
def numerify_df(data):
    for c in data.columns:
        try:
            data[c] = data[c].astype(float)
        except:
            data = data.drop(columns=c)
    return data

def get_elements(df):
    new_cols = []
    for c in df.columns:
        if len(c)<3:
            if c is not 'A5' and c is not 'Rm':
                new_cols.append(c)
            
    return df[new_cols]


In [6]:
def get_merged_data():
    #Load initial data
    new_data = pd.read_csv('../data/final_data/02_23_newdata.csv').drop(columns='Unnamed: 0')
    new_data['Category'] = 'Stick'
    print("NEW DATA" + str(new_data.shape))
    linc_data = pd.read_csv('../data/final_data/02_23_lincdata.csv').drop(columns='Unnamed: 0')
    print("LINC DATA" + str(linc_data.shape))
    weld_data = pd.read_csv('../data/final_data/02_23_welddata.csv').drop(columns='Unnamed: 0')
    print("CEWELD DATA" + str(weld_data.shape))
    cit_data = pd.read_csv('../data/cit_data/200124_cit_data.csv').drop(columns='Unnamed: 0')
    print("CIT DATA" + str(cit_data.shape))
    
    take_smaller = lambda s1, s2: s1 if s1.sum() < s2.sum() else s2
    new_data['UTS'] = new_data['UTS'].combine(new_data['UTS(Kg/mm2)'], take_smaller)
    new_data = new_data.drop(columns={'UTS(Kg/mm2)','Elong %'})
    
    
    #line up datasets and merge them
    new_data.columns = ['Diffusible Hydrogen', 'Moisture', 'Yield strength', 'Tensile', 'Hardness',
           'Ferrite (Fn)', 'Hardness_Scale', 'V', 'C', 'Cr', 'Mn', 'Mo', 'Ni',
           'P ', 'S ', 'Si', 'Cb', 'Cu', 'Category', 'Elongation']

    linc_data.columns = ['As', 'C', 'Charpy', 'Cr', 'Cu', 'Diffusible Hydrogen', 'Elongation',
           'Lateral', 'Mn', 'Mn + Ni + Cr + Mo + V', 'Mo', 'N', 'Nb', 'Ni',
           'Ni+Mn', 'P', 'P S', 'S', 'S Ni', 'SN', 'Si', 'Si P', 'Sn', 'Tensile',
           'V', 'Yield strength', 'aws', 'conformances', 'ind', 'key features',
           'requirements', 'typic_results', 'typical applications', 'Category',
           'name', 'url']

    new_data = new_data.append(linc_data, ignore_index=True, sort=False)
    new_data = new_data.append(weld_data, ignore_index=True, sort=False)
    new_data = new_data.append(cit_data, ignore_index=True, sort=False)
    
    
    #Merging Stick Columns
    sticks = ['7018.0', '  6013 (REST)', '  LOTHERME', '  LOW ALLOY(SPL)', '  ST. STEELS', 'stainless-high-alloy', 'SMAW Stick Electrodes', 'stick-electrodes', '  6013 (NORMA-V)']

    for s in sticks:
        new_data['Category'] = new_data['Category'].replace(s,'Stick')
    
    #Identifying Stick and non-Stick
    uns = new_data['Category'].unique()[1:]
    for u in uns:
        new_data['Category'] = new_data['Category'].replace(u,"Others")
        new_data['Category'].unique()
    uns = new_data['Category'].unique()

    i = 0
    for u in uns:
        new_data['Category'] = new_data['Category'].replace(u,1-i)
        i = i+1
    
    #1 means stick, 0 means other
    new_data = new_data.drop(columns={'ind'})
    new_data = new_data.dropna(axis=1, thresh=100)
    
    
    return new_data

#If true it will only return stick dataset
def get_ml_data(stick):
    #Get merged dataset
    df = get_merged_data()
    
    
    #Get training dataset
    df = numerify_df(df).astype(float)
    columns = list(df.columns)
    df = df.to_numpy()
    
    #Getting rid of non-sticks
    df[df==0] = np.nan
    if stick:
        df = df[pd.notnull(df[:,columns.index("Category")])]
    
    tmp = pd.DataFrame(df, columns = columns)
    elems = get_elements(tmp).drop(columns={'A5', 'Rm'})
    elem_cols = elems.columns
    elems = numerify_df(elems).astype(float).to_numpy()
    elems[elems==0] = np.nan
    
    
        
    return df, elems, elem_cols, columns

#Getting all of the columns of parameters to predict
def get_pred_cols():
    pred_cs = ['Rm', 'A5']
    for c in columns:
        if len(c)>2:
            pred_cs.append(c)
    return pred_cs

In [7]:
#Methods for pre-examining data

def get_model_err(df,prop, model):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    #tmp[tmp==0] = np.nan
    #tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    lin_reg = model
    lin_reg.fit(X_train, y_train)
    pred = lin_reg.predict(X_test)
    err = sqrt(mean_squared_error(y_test, pred))
    return err

def get_model_residual(df,prop, model):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    #tmp[tmp==0] = np.nan
    #tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    #Trying data transformation
    X_train = X_train+0.0000000000000001
    X_test = X_test+0.0000000000000001
    X_train = np.power(X_train,-.2)
    X_test = np.power(X_test,-.2)
    lin_reg = ResidualsPlot(model)
    lin_reg.fit(X_train, y_train)
    lin_reg.score(X_test, y_test)
    return lin_reg

def try_model(model):
    a = []
    for c in get_pred_cols():
        err = get_model_err(df, c,model)
        a.append(err)
    i = 0
    for c in a:
        print(columns[i] +" "+str(c))
        i+=1
    return a
        
def get_res_plots(model):
    a = []
    for c in get_pred_cols():
        err = get_model_residual(df, c,model)
        print(columns[i])
        err.show()
        a.append(err)


In [8]:
df, elems, elem_cols, columns = get_ml_data(True)

NEW DATA(1509, 22)
LINC DATA(944, 36)
CEWELD DATA(678, 51)
CIT DATA(1650, 75)


In [10]:
try_model(RandomForestRegressor())

/home/prasann/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Diffusible Hydrogen 83.77677674663482
Moisture 2.6074987141661166
Yield strength 0.4946381614364334
Tensile 0.015895286243181157
Ferrite (Fn) 81.76462319130242
V 94.79607525540142
C 0.8729046887868843
Cr 0.0
Mn 4.575317728629353
Mo 20.853747011842042
Ni 0.12641470039089994
P  26.306324853022083
S  27.879341841356997
Si 17.436154640182668
Cb 8.557113727405342
Cu 0.17079524472680122
Category 8497.952727020938
Elongation 0.0
Charpy 0.0
P 0.0
S 0.0
A5 0.0
Co 0.0
DIN_W 0.0
Fe 0.0


[83.77677674663482,
 2.6074987141661166,
 0.4946381614364334,
 0.015895286243181157,
 81.76462319130242,
 94.79607525540142,
 0.8729046887868843,
 0.0,
 4.575317728629353,
 20.853747011842042,
 0.12641470039089994,
 26.306324853022083,
 27.879341841356997,
 17.436154640182668,
 8.557113727405342,
 0.17079524472680122,
 8497.952727020938,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
t = try_model(svm.SVR(kernel = 'linear', C=1000))

In [117]:
try_model(XGBRegressor())

[21:20:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:20:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:20:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:20:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:20:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:20:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Diffusible Hydrogen 0.025769667260061555
Moisture 43.584973200786564
Yield strength 76.17510628210505
Tensile 33.75390072534243
Hardness 0.2508630262547021
Ferrite (Fn) 5.326949167451765


[0.025769667260061555,
 43.584973200786564,
 76.17510628210505,
 33.75390072534243,
 0.2508630262547021,
 5.326949167451765]

In [8]:

#Hyperopt code sample 1

# XGB parameters
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
    'silent':True,
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# LightGBM parameters
lgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
lgb_fit_params = {
    'eval_metric': 'l2',
    'early_stopping_rounds': 10,
    'verbose': False
}
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# CatBoost parameters
ctb_reg_params = {
    'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'max_depth':         hp.choice('max_depth',         np.arange(5, 16, 1, dtype=int)),
    'colsample_bylevel': hp.choice('colsample_bylevel', np.arange(0.3, 0.8, 0.1)),
    'n_estimators':      100,
    'eval_metric':       'RMSE',
}
ctb_fit_params = {
    'early_stopping_rounds': 10,
    'verbose': False
}
ctb_para = dict()
ctb_para['reg_params'] = ctb_reg_params
ctb_para['fit_params'] = ctb_fit_params
ctb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

In [9]:

class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def lgb_reg(self, para):
        reg = lgb.LGBMRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def ctb_reg(self, para):
        reg = ctb.CatBoostRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [10]:
def get_model_hpt_err(df,prop):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    #tmp[tmp==0] = np.nan
    #tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    
    obj = HPOpt(X_train, X_test, y_train, y_test)
    
    xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)
    #lgb_opt = obj.process(fn_name='lgb_reg', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)
    #ctb_opt = obj.process(fn_name='ctb_reg', space=ctb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

    #lin_reg = model
    #lin_reg.fit(X_train, y_train)
    pred = xgb_opt[0]
    #err = mean_squared_error(y_test, pred)
    #return err
    print(xgb_opt)
    
def try_hpt_model():
    a = []
    cats = [1, 2, 3, 4, 17, 18]
    for i in cats:
        get_model_hpt_err(df, columns[i])
        #a.append(err)
    i = 0
    for c in a:
        if c<1:
            print(columns[i] +" "+str(c))
        i+=1
    return a

In [11]:
print(get_model_hpt_err(df, "Charpy"))

100%|██████████| 100/100 [00:05<00:00, 17.55trial/s, best loss: 1.1920928955078125e-07]
({'colsample_bytree': 0, 'learning_rate': 4, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.8977608652442637}, <hyperopt.base.Trials object at 0x7f827239a8d0>)
None


In [14]:
#trying out Gaussian Processes
import sklearn.gaussian_process as gp
kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0, (1e-3, 1e3))
model = gp.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1, normalize_y=True)
get_model_err(df, "Charpy",model)

KeyboardInterrupt: 

In [17]:
#Hyperparameter optimization code 2

models = {
    'logistic_regression': LogisticRegression,
    'rf': RandomForestRegressor,
    'knn': KNeighborsRegressor, 'svr': SVR,
    'xgb': XGBRegressor
}


def search_space(model):

    model = model.lower()
    space = {}

    if model == 'knn':
        space = {'n_neighbors': hp.choice('n_neighbors', range(1, 100)),


                 }

    elif model == 'xgb':
        space = {'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
    'silent':True,
                 }

    elif model == 'svr':
        space = {
            'C': hp.uniform('C', 0, 20),
            'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
            'gamma': hp.uniform('gamma', 0, 20),
            'scale': hp.choice('scale', [0, 1]),
            'normalize': hp.choice('normalize', [0, 1]),
        }

    elif model == 'logistic_regression':
        space = {
            'warm_start': hp.choice('warm_start', [True, False]),
            'fit_intercept': hp.choice('fit_intercept', [True, False]),
            'tol': hp.uniform('tol', 0.00001, 0.0001),
            'C': hp.uniform('C', 0.05, 3),
            'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
            'max_iter': hp.choice('max_iter', range(100, 1000)),
            'scale': hp.choice('scale', [0, 1]),
            'normalize': hp.choice('normalize', [0, 1]),
            'multi_class': 'auto',
            'class_weight': 'balanced'
        }
    elif model == 'rf':
        space = {'max_depth': hp.choice('max_depth', range(1, 20)),
                 'max_features': hp.choice('max_features', range(1, 10)),
                 'n_estimators': hp.choice('n_estimators', range(10, 50)),
                 'criterion': hp.choice('criterion', ["gini", "entropy"]),
                 }
    return space

In [14]:
from hyperopt import space_eval
kf = KFold(n_splits=5, random_state=42)
curmodel = XGBRegressor
curvar = "Category"
dataset = 1
def get_data(prop):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    tmp[tmp==0] = np.nan
    tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.333, random_state=42, shuffle=True)
    return x_val, x_test, x_train, y_val, y_test, y_train

def objective(space):
    #print(space)
    clf = curmodel(
                           **space, objective = 'reg:squarederror'
                            )

    
    x_val, x_test, x_train, y_val, y_test, y_train = get_data(curvar)
    
    eval_set  = [( x_train, y_train), ( x_val, y_val)]

    clf.fit(x_train, y_train,
            eval_set=eval_set, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False,)

    pred = clf.predict(x_test)
    mse_scr = -cross_val_score(clf, x_train, y_train, cv=kf, scoring="neg_mean_squared_error", n_jobs=-1).mean()
    #print ("SCORE:", np.sqrt(mean_squared_error(y_test, pred)))
    #change the metric if you like
    pred = clf.predict(x_test)
    score = np.sqrt(mean_squared_error(y_test, pred))
    return {'loss':np.sqrt(mse_scr), 'status': STATUS_OK, 'sco':score }

def get_model(space):
    #print(space)
    clf = curmodel(
                           **space, objective = 'reg:squarederror'
                            )

    
    x_val, x_test, x_train, y_val, y_test, y_train = get_data(curvar)
    
    eval_set  = [( x_train, y_train), ( x_val, y_val)]

    clf.fit(x_train, y_train,
            eval_set=eval_set, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False,)

    pred = clf.predict(x_test)
    mse_scr = -cross_val_score(clf, x_train, y_train, cv=kf, scoring="neg_mean_squared_error", n_jobs=-1).mean()
    #print ("SCORE:", np.sqrt(mean_squared_error(y_test, pred)))
    #change the metric if you like
    pred = clf.predict(x_test)
    score = np.sqrt(mean_squared_error(y_test, pred))
    return clf

def run_opt(prop, space):

    curmodel = models[space]
    curvar = prop
    trials = Trials()
    
    #TODO change max_evals to 100
    best = fmin(fn=objective,
                space=search_space(space),
                algo=tpe.suggest,
                max_evals=10,
                trials=trials,
               return_argmin=False)
    
    return trials.best_trial['result']['loss'], trials.best_trial['result']['sco'], best

def get_opt_model(prop, space):

    #TODO Change max evals to 100
    curmodel = models[space]
    curvar = prop
    trials = Trials()
    best = fmin(fn=objective,
                space=search_space(space),
                algo=tpe.suggest,
                max_evals=10,
                trials=trials,
               return_argmin=False)
    
    model = get_model(best)
    
    return model

def get_dataset_(prop):
    tmp = np.concatenate([df, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    tmp[tmp==0] = np.nan
    tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    return X

In [20]:
val, los, mod = run_opt("Charpy", 'logistic_regression')

100%|██████████| 10/10 [00:02<00:00,  4.50trial/s, best loss: 1.3423093046464597e-05]


In [15]:
#Code for getting benchmarks using hyperopt code 2
def get_benchmarks():
    d = pd.DataFrame()
    a = {}
    a['data'] = dataset
    for m in models.keys():
        print(m)
        a['model'] = m
        for c in get_pred_cols():
            print(c)
            a['column'] = c
            a['validation'], a['test'], a['params'] = run_opt(c, m)
            d = d.append(a, ignore_index=True, sort=False)
    return d

def get_models(prop):
    d = []
    for c in get_pred_cols():
        d.append(get_opt_model(c, prop))
    return d

In [18]:
#TODO Fix strange bug
xgb_models = get_models('xgb')

 10%|█         | 1/10 [00:00<00:01,  5.81trial/s, best loss: -0.0]


KeyboardInterrupt: 

In [166]:
d = get_benchmarks()
d.to_csv('../benchmarks/benchmark_2.csv')

logistic_regression
Rm
100%|██████████| 10/10 [00:02<00:00,  4.72trial/s, best loss: 1.33514404296875e-05]
A5
100%|██████████| 10/10 [00:03<00:00,  3.30trial/s, best loss: 1.33514404296875e-05]
Diffusible Hydrogen
100%|██████████| 10/10 [00:02<00:00,  3.43trial/s, best loss: 1.33514404296875e-05]
Moisture
100%|██████████| 10/10 [00:02<00:00,  4.98trial/s, best loss: 1.33514404296875e-05]
Yield strength
100%|██████████| 10/10 [00:02<00:00,  3.39trial/s, best loss: 1.33514404296875e-05]
Tensile
100%|██████████| 10/10 [00:02<00:00,  3.45trial/s, best loss: 1.33514404296875e-05]
Hardness
100%|██████████| 10/10 [00:02<00:00,  4.77trial/s, best loss: 1.33514404296875e-05]
Ferrite (Fn)
100%|██████████| 10/10 [00:02<00:00,  3.46trial/s, best loss: 1.33514404296875e-05]
Category
100%|██████████| 10/10 [00:02<00:00,  3.46trial/s, best loss: 1.33514404296875e-05]
Elongation
100%|██████████| 10/10 [00:01<00:00,  5.05trial/s, best loss: 1.33514404296875e-05]
Charpy
100%|██████████| 10/10 [00:02<00: